In [25]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

In [26]:
url = 'https://www.soccerbase.com/teams/head_to_head.sd?team_id=142&team2_id=1718'
response = requests.get(url)
response.status_code

200

In [27]:
soup = bs(response.text)


In [28]:
league = soup.find('td', class_='first tournament')
league.text

'Premier League'

In [29]:
date=soup.find('td',class_='dateTime')
date.text

'Sa 01Jan 2022'

In [30]:
hometeam=soup.find('td',class_='team homeTeam')
hometeam.text


'Arsenal'

In [31]:
awayteam=soup.find('td',class_='team awayTeam')
awayteam.text

'Man City'

In [32]:
home_score=soup.find('td',class_='score')
home_score.text[0]

'1'

In [33]:
away_score=soup.find('td',class_='score')
away_score.text[-1]

'2'

In [34]:
def get_details(soup):
  league = soup.find('td', class_='first tournament')
  league.text

  date=soup.find('td',class_='dateTime')
  date.text

  hometeam=soup.find('td',class_='team homeTeam')
  hometeam.text

  awayteam=soup.find('td',class_='team awayTeam')
  awayteam.text

  home_score=soup.find('td',class_='score')
  home_score.text[0]

  away_score=soup.find('td',class_='score')
  away_score.text[-1]
 
  return league.text, date.text, hometeam.text,awayteam.text, home_score.text[0],away_score.text[-1]

In [35]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:
        return bs(resp.text)
    else: return None


def get_books(url):
    """Extact details from all the book tags"""
    soup = get_soup(url)
    book_tags = soup.find_all('tbody')

    books = []
    for book_tag in book_tags:
        books.append(get_details(book_tag))

    return books

In [36]:
def get_soup(url):
    """Takes URL and returns a soup object"""
    resp = requests.get(url)
    if resp.status_code == 200:
        return bs(resp.text)
    else: return None


def get_books(url):
    """Extact details from all the book tags"""
    soup = get_soup(url)
    book_tags = soup.find_all('tr', class_='match')

    books = []
    for book_tag in book_tags:
        books.append(get_details(book_tag))

    return books

In [37]:
url = 'https://www.soccerbase.com/teams/head_to_head.sd?team_id=142&team2_id=1718'
books = get_books(url)
len(books)

205

In [38]:
import pandas as pd

def get_all_books(page = 0):
    books = []
    for i in range(1, page+1):
        ## This is how the url changes with every page
        url = 'https://www.soccerbase.com/teams/head_to_head.sd?team_id=142&team2_id=1718'
        soup = get_soup(url)
        if soup:    
            book_tags = soup.find_all('tr', class_='match')

            for book_tag in book_tags:
                books.append(get_details(book_tag))
            
    books = pd.DataFrame(books, columns=['league', 'date','hteam', 'ateam', 'hscore', 'ascore'])
    return books

In [39]:
df = get_all_books(1)
df.head(5)

,league,date,hteam,ateam,hscore,ascore
0,Premier League,Sa 01Jan 2022,Arsenal,Man City,1,2
1,Premier League,Sa 28Aug 2021,Man City,Arsenal,5,0
2,Premier League,Su 21Feb 2021,Arsenal,Man City,0,1
3,EFL Cup,Tu 22Dec 2020,Arsenal,Man City,1,4
4,Premier League,Sa 17Oct 2020,Man City,Arsenal,1,0


In [40]:
df1=df.copy()
df2=df.copy()


In [41]:
df2.loc[df2['hscore']>df2['ascore'],'hteam']

1      Man City
4      Man City
5       Arsenal
6      Man City
8      Man City
         ...   
196    Man City
199    Man City
201    Man City
202     Arsenal
204     Arsenal
Name: hteam, Length: 99, dtype: object

In [42]:
df2 = df1[['hteam', 'ateam',	'hscore', 'ascore']]
df2['Winner'] = 'Draw'
df2.loc[df2['hscore']>df2['ascore'],'Winner'] = df2.loc[df2['hscore']>df2['ascore'],'hteam']
df2.loc[df2['hscore']<df2['ascore'],'Winner'] = df2.loc[df2['hscore']<df2['ascore'],'ateam']
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,hteam,ateam,hscore,ascore,Winner
0,Arsenal,Man City,1,2,Man City
1,Man City,Arsenal,5,0,Man City
2,Arsenal,Man City,0,1,Man City
3,Arsenal,Man City,1,4,Man City
4,Man City,Arsenal,1,0,Man City
...,...,...,...,...,...
200,Arsenal,Man City,0,1,Man City
201,Man City,Arsenal,4,1,Man City
202,Arsenal,Man City,4,2,Arsenal
203,Man City,Arsenal,0,1,Arsenal


In [43]:
df1 = df.copy()
df1['Winner'] = 0

df1.loc[df1['hscore']>df1['ascore'],'Winner'] = 1

df1.loc[df1['hscore']<df1['ascore'],'Winner'] = -1

df1

,league,date,hteam,ateam,hscore,ascore,Winner
0,Premier League,Sa 01Jan 2022,Arsenal,Man City,1,2,-1
1,Premier League,Sa 28Aug 2021,Man City,Arsenal,5,0,1
2,Premier League,Su 21Feb 2021,Arsenal,Man City,0,1,-1
3,EFL Cup,Tu 22Dec 2020,Arsenal,Man City,1,4,-1
4,Premier League,Sa 17Oct 2020,Man City,Arsenal,1,0,1
...,...,...,...,...,...,...,...
200,English Div 2 (old),Sa 07Sep 1895,Arsenal,Man City,0,1,-1
201,English Div 2 (old),Sa 15Dec 1894,Man City,Arsenal,4,1,1
202,English Div 2 (old),Sa 29Sep 1894,Arsenal,Man City,4,2,1
203,English Div 2 (old),Sa 30Dec 1893,Man City,Arsenal,0,1,-1


In [44]:
# if df1.loc[df1['hscore']>df1['ascore']]:
#   df1['Winner'] = 1

# if df1.loc[df1['hscore']<df1['ascore']]:
#   df1['Winner'] = 1

# else:
#   df1['Winner'] = 0

In [45]:
df = df.astype({'hscore':'float','ascore':'float'})
df['winner']=''
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   league  205 non-null    object 
 1   date    205 non-null    object 
 2   hteam   205 non-null    object 
 3   ateam   205 non-null    object 
 4   hscore  205 non-null    float64
 5   ascore  205 non-null    float64
 6   winner  205 non-null    object 
dtypes: float64(2), object(5)
memory usage: 11.3+ KB


In [46]:
h=df.hscore
a=df.ascore
# x=(df[df['hscore']>df['ascore']])
x=df['hscore']>df['ascore']
df.loc[:,'winner']=x
# if df[h>a]:
#   df.winner='win'
# else:
#   df.winner='Draw'
# for i in df['winner']:
#   if i==False:
#     df['winner']=df['winner'].replace('False',"df['ateam']")
#   else:
#     df['winner']=df['winner'].replace('True',"df['hteam']")
df
df['winner']

0      False
1       True
2      False
3      False
4       True
       ...  
200    False
201     True
202     True
203    False
204     True
Name: winner, Length: 205, dtype: bool

In [47]:
#df['date'] = df['date'].replace(['Sa','Su','Mo','Tu','We','Th','Fr'],['Saturday','Sunday','Monday','Tuesday','Wednesday','Thursday','Friday'])

In [48]:
df.date.str.split('')

0      [, S, a,  , 0, 1, J, a, n,  , 2, 0, 2, 2, ]
1      [, S, a,  , 2, 8, A, u, g,  , 2, 0, 2, 1, ]
2      [, S, u,  , 2, 1, F, e, b,  , 2, 0, 2, 1, ]
3      [, T, u,  , 2, 2, D, e, c,  , 2, 0, 2, 0, ]
4      [, S, a,  , 1, 7, O, c, t,  , 2, 0, 2, 0, ]
                          ...                     
200    [, S, a,  , 0, 7, S, e, p,  , 1, 8, 9, 5, ]
201    [, S, a,  , 1, 5, D, e, c,  , 1, 8, 9, 4, ]
202    [, S, a,  , 2, 9, S, e, p,  , 1, 8, 9, 4, ]
203    [, S, a,  , 3, 0, D, e, c,  , 1, 8, 9, 3, ]
204    [, S, a,  , 1, 1, N, o, v,  , 1, 8, 9, 3, ]
Name: date, Length: 205, dtype: object